In [1]:
!pip install py2neo


  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
Using legacy 'setup.py install' for neotime, since package 'wheel' is not installed.
  Attempting uninstall: six
    Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.3
    Uninstalling prompt-toolkit-3.0.3:
      Successfully uninstalled prompt-toolkit-3.0.3
    Running setup.py install for neotime: started
    Running setup.py install for neotime: finished with status 'done'


You should consider upgrading via the 'c:\users\maxim\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.


In [2]:


# import librairies
import numpy as np 
import pandas as pd
from py2neo import Graph
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import time

ModuleNotFoundError: No module named 'pandas'

In [10]:
# load the data
df = pd.read_csv('netflix_titles.csv')
df["date_added"] = pd.to_datetime(df['date_added'])
df['year'] = df['date_added'].dt.year
df['month'] = df['date_added'].dt.month
df['day'] = df['date_added'].dt.day

df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year,month,day
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,2019.0,9.0,9.0
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,2016-09-09,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,2016.0,9.0,9.0
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,2018-09-08,2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob...",2018.0,9.0,8.0
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,2018-09-08,2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...,2018.0,9.0,8.0
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,2017-09-08,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,2017.0,9.0,8.0


In [11]:
#Connect to neo4j sandbox
ip="54.160.68.122"
port="32924"
pwd="inventions-evacuation-tumbles"
graph = Graph("bolt://"+ip+":"+port, auth=("neo4j", pwd))

In [16]:
LOAD_DATA = True

def run_request(request,LOAD_DATA=True):
    if (LOAD_DATA):
        start_time = time.time()
        print(graph.run(request)) 
        exec_time = time.time() - start_time
        print(f'-- {exec_time} seconds --')

In [17]:
# DELETE ALL NODES AND RELATIONSHIPS
request_delete ="""
MATCH (n) DETACH DELETE n;
"""
run_request(request_delete,LOAD_DATA)

(No data)
-- 0.5247945785522461 seconds --


In [ ]:
# CREATE MOVIES

request ="""
CREATE (m:Movie {id: {show_id},title:{title}})
SET 
m.director = {director},
m.country = {country},
m.date_str = {date_added}, 
m.release_year = {release_year},
m.rating = {rating},
m.duration = {duration} ,
m.listed_in = {listed_in} ,
m.description = {description},
m.cast={cast},
m.year = {year},
m.month = {month},
m.day = {day},
m.type = {type_movie}
"""
start_time = time.time()
count=df.shape[0]
if (LOAD_DATA):
    for index, row in df.iterrows():
        graph.run(request,
                  show_id=row['show_id'],
                  title=row['title'],
                  director=None if pd.isnull(row['director']) else row['director'],
                  country=None if pd.isnull(row['country']) else row['country'],
                  date_added= None if pd.isnull(row['date_added']) else row['date_added'].strftime('%Y-%m-%d'),
                  release_year=None if pd.isnull(row['release_year']) else row['release_year'],
                  rating=None if pd.isnull(row['rating']) else row['rating'],          
                  duration=None if pd.isnull(row['duration']) else row['duration'],
                  listed_in=None if pd.isnull(row['listed_in']) else row['listed_in'],
                  description=None if pd.isnull(row['description']) else row['description'],
                  cast= None if pd.isnull(row['cast']) else row['cast'],
                  year=None if pd.isnull(row['year']) else row['year'],
                  month=None if pd.isnull(row['month']) else row['month'],
                  day=None if pd.isnull(row['day']) else row['day'],
                  type_movie=None if pd.isnull(row['type']) else row['type']
                  )
        if (index%500==0):
            exec_time = time.time() - start_time
            print(f'{index}/{df.shape[0]} done ... -- {exec_time} seconds --')
        
    exec_time = time.time() - start_time
    print(f' ... End ... -- {exec_time} seconds --')        

0/6234 done ... -- 0.676306962966919 seconds --
500/6234 done ... -- 159.71222925186157 seconds --
1000/6234 done ... -- 314.0621554851532 seconds --
1500/6234 done ... -- 464.5960121154785 seconds --
2000/6234 done ... -- 620.7064392566681 seconds --
2500/6234 done ... -- 764.8139653205872 seconds --
3000/6234 done ... -- 912.9551634788513 seconds --
3500/6234 done ... -- 1063.6301403045654 seconds --
4000/6234 done ... -- 1246.2718269824982 seconds --
4500/6234 done ... -- 1507.5887825489044 seconds --


In [15]:
# CREATE  PERSONS
request ="""
MATCH (m:Movie)
WHERE m.cast IS NOT NULL
WITH m
UNWIND split(m.cast, ',') AS actor
MERGE (p:Person {name: trim(actor)})
MERGE (p)-[r:ACTED_IN]->(m);
"""    
run_request(request,LOAD_DATA)   